In [1]:
import pandas as pd
from fuzzywuzzy import fuzz
from itertools import chain
import numpy as np

# Read in the datasets
left_df = pd.read_csv("left_dataset.csv")
right_df = pd.read_csv("right_dataset.csv")

# Make copies of the original dataframes for cleaning
left_df_cleaned = left_df.copy()
right_df_cleaned = right_df.copy()

# Replace "N/A" values in the "address" column with NaN
left_df_cleaned['address'] = left_df_cleaned['address'].str.replace(r'\bN/A\b', '').replace('', pd.NA)
right_df_cleaned['address'] = right_df_cleaned['address'].str.replace(r'\bN/A\b', '').replace('', pd.NA)

# Remove trailing commas from the "address" column
left_df_cleaned['address'] = left_df_cleaned['address'].str.rstrip(',')
right_df_cleaned['address'] = right_df_cleaned['address'].str.rstrip(',')

# Convert strings to lowercase/uppercase
left_df_cleaned['address'] = left_df_cleaned['address'].str.lower()
right_df_cleaned['address'] = right_df_cleaned['address'].str.lower()
left_df_cleaned['name'] = left_df_cleaned['name'].str.lower()
right_df_cleaned['name'] = right_df_cleaned['name'].str.lower()
left_df_cleaned['city'] = left_df_cleaned['city'].str.lower()
right_df_cleaned['city'] = right_df_cleaned['city'].str.lower()
left_df_cleaned['state'] = left_df_cleaned['state'].str.upper()
right_df_cleaned['state'] = right_df_cleaned['state'].str.upper()

## clean zip codes
# clean zip code in left dataset
left_df_cleaned['zip_code'] = left_df_cleaned['zip_code'].str.slice(0, 5)

# clean zip code in right dataset
right_df_cleaned['postal_code'] = right_df_cleaned['postal_code'].fillna('')
right_df_cleaned['postal_code'] = right_df_cleaned['postal_code'].astype(str)
right_df_cleaned['postal_code'] = right_df_cleaned['postal_code'].str.replace(r'\.\d+', '')


C:\Users\mikes\AppData\Local\Temp\ipykernel_23136\266909364.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  left_df_cleaned['address'] = left_df_cleaned['address'].str.replace(r'\bN/A\b', '').replace('', pd.NA)
C:\Users\mikes\AppData\Local\Temp\ipykernel_23136\266909364.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  right_df_cleaned['address'] = right_df_cleaned['address'].str.replace(r'\bN/A\b', '').replace('', pd.NA)
C:\Users\mikes\AppData\Local\Temp\ipykernel_23136\266909364.py:39: FutureWarning: The default value of regex will change from True to False in a future version.
  right_df_cleaned['postal_code'] = right_df_cleaned['postal_code'].str.replace(r'\.\d+', '')


In [2]:
def matcher(left_df, right_df):
    
    left_id = []
    right_id = []
    score = []
    for _, lrow in left_df.iterrows():
        for _, rrow in right_df.iterrows():
            score1 = fuzz.token_set_ratio(lrow['name'], rrow['name']) / 100
            score2 = fuzz.token_set_ratio(lrow['address'], rrow['address']) / 100
            
            if min(score1,score2)>= 0.8:
                left_id.append(lrow['business_id'])
                right_id.append(rrow['entity_id'])
                score.append(min(score1,score2))
    return [left_id ,right_id, score]

In [3]:
def approximate_match(left_df,right_df):
    postal_codes = set(left_df['zip_code'])&set(right_df['postal_code'])
    left_id = []
    right_id = []
    score = []
    
    left_group = left_df.groupby('zip_code', as_index = True)
    right_group = right_df.groupby('postal_code', as_index = True)
    
    for i in postal_codes:
        left = left_group.get_group(i)
        right = right_group.get_group(i)
        matches = matcher(left,right)
        left_id.extend(matches[0])
        right_id.extend(matches[1])
        score.extend(matches[2])
    return [left_id ,right_id, score]

In [4]:
left_head = left_df_cleaned.iloc[60000:70000]
right_head = right_df_cleaned.iloc[40000:50000]

In [22]:
result = approximate_match(left_df_cleaned,right_df_cleaned)

In [24]:
len(result)

3

In [25]:
len(result[1])

10664

In [34]:
result

[[214,
  494,
  1423,
  1779,
  1898,
  2223,
  2317,
  2437,
  2596,
  2596,
  2736,
  3011,
  3661,
  3717,
  3824,
  3825,
  3911,
  3912,
  3957,
  4565,
  4810,
  5514,
  5604,
  6609,
  7358,
  7611,
  7658,
  7757,
  7800,
  7801,
  7813,
  8128,
  8133,
  8421,
  9346,
  9640,
  9936,
  10073,
  10293,
  10444,
  10456,
  10592,
  10870,
  10871,
  11168,
  11725,
  12154,
  12165,
  12449,
  12990,
  13793,
  13930,
  13931,
  15100,
  15945,
  16829,
  16984,
  17042,
  17386,
  17387,
  17406,
  17452,
  17870,
  20745,
  20896,
  23701,
  24472,
  25016,
  25072,
  26119,
  27001,
  27148,
  28028,
  1679,
  8916,
  9791,
  14843,
  14989,
  23820,
  23821,
  26596,
  4208,
  4216,
  4957,
  6027,
  6393,
  7397,
  9088,
  9774,
  10769,
  10770,
  12731,
  15753,
  19436,
  20418,
  20786,
  21527,
  22260,
  22475,
  23036,
  24457,
  26454,
  27141,
  56620,
  59057,
  60552,
  62722,
  62934,
  66267,
  66393,
  66455,
  66735,
  66959,
  68281,
  68575,
  70277,
  7030

In [39]:
left_df.loc[left_df['business_id'] == result[0][10050]]

,business_id,name,address,city,state,zip_code,size
10566,10567,FIVE STAR AUTO SALES OF TAMPA INC,11028 N Florida Ave,Tampa,FL,33612-6636,3.0


In [40]:
right_df.loc[right_df['entity_id'] == result[1][10050]]

,entity_id,name,address,city,state,postal_code,categories
47753,47754,Five Star Auto Sales Tampa,10409 N Florida Ave,Tampa,FL,33612.0,"Car Dealers, Automotive"


In [12]:
len(left_df_cleaned.drop_duplicates(subset = ['name','address'],keep='first'))

93231

In [13]:
len(left_df_cleaned)

98509

In [14]:
len(right_df_cleaned.drop_duplicates(subset = ['name','address'],keep='first'))

94391

In [15]:
len(right_df_cleaned)

94585